In [2]:
import pandas as pd
import plotly.graph_objects as go
import numpy as np
import datetime

In [3]:
# # https://catalog.data.gov/dataset/alzheimers-disease-and-healthy-aging-data/resource/b2443e45-68aa-4004-b97a-e62c4c28502f
# df= pd.read_csv('https://catalog.data.gov/dataset/real-estate-sales-2001-2018')
# display(df)

In [4]:
# df.astype(str).to_parquet(path= 'data_frame.gzip', compression='gzip')
df= pd.read_parquet('data_frame.gzip')

In [5]:
df[['Serial Number', 'List Year']]= df[['Serial Number', 'List Year']].astype(int)
df[['Assessed Value', 'Sale Amount', 'Sales Ratio']]= df[['Assessed Value', 'Sale Amount', 'Sales Ratio']].astype(float)
# df['Date Recorded']= pd.to_datetime(df['Date Recorded'], format= '%d/%M/%Y')

df

,Serial Number,List Year,Date Recorded,Town,Address,Assessed Value,Sale Amount,Sales Ratio,Property Type,Residential Type,Non Use Code,Assessor Remarks,OPM remarks,Location
0,2020177,2020,04/14/2021,Ansonia,323 BEAVER ST,133000.0,248400.0,0.535400,Residential,Single Family,nan,nan,nan,POINT (-73.06822 41.35014)
1,2020225,2020,05/26/2021,Ansonia,152 JACKSON ST,110500.0,239900.0,0.460600,Residential,Three Family,nan,nan,nan,nan
2,2020348,2020,09/13/2021,Ansonia,230 WAKELEE AVE,150500.0,325000.0,0.463000,Commercial,nan,nan,nan,nan,nan
3,2020090,2020,12/14/2020,Ansonia,57 PLATT ST,127400.0,202500.0,0.629100,Residential,Two Family,nan,nan,nan,nan
4,200500,2020,09/07/2021,Avon,245 NEW ROAD,217640.0,400000.0,0.544100,Residential,Single Family,nan,nan,nan,nan
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
997208,190272,2019,06/24/2020,New London,4 BISHOP CT,60410.0,53100.0,1.137665,Single Family,Single Family,14 - Foreclosure,nan,nan,nan
997209,190284,2019,11/27/2019,Waterbury,126 PERKINS AVE,68280.0,76000.0,0.898400,Single Family,Single Family,25 - Other,PRIVATE SALE,nan,nan
997210,190129,2019,04/27/2020,Windsor Locks,19 HATHAWAY ST,121450.0,210000.0,0.578300,Single Family,Single Family,nan,nan,nan,nan
997211,190504,2019,06/03/2020,Middletown,8 BYSTREK DR,203360.0,280000.0,0.726300,Single Family,Single Family,nan,nan,nan,nan


In [6]:
df= df[df['Date Recorded'].apply(lambda x: len(x)) == 10]


df['year']= df['Date Recorded'].apply(lambda x: x[6:])
df['month']= df['Date Recorded'].apply(lambda x: x[3:5])
df['day']= df['Date Recorded'].apply(lambda x: x[:2])




C:\Users\jmart\AppData\Local\Temp\ipykernel_11204\608499965.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['year']= df['Date Recorded'].apply(lambda x: x[6:])
C:\Users\jmart\AppData\Local\Temp\ipykernel_11204\608499965.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['month']= df['Date Recorded'].apply(lambda x: x[3:5])
C:\Users\jmart\AppData\Local\Temp\ipykernel_11204\608499965.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row

In [7]:
df= df[(df['month'].astype(int) <= 12) & (df['day'].astype(int) != 0)]


In [8]:
df

,Serial Number,List Year,Date Recorded,Town,Address,Assessed Value,Sale Amount,Sales Ratio,Property Type,Residential Type,Non Use Code,Assessor Remarks,OPM remarks,Location,year,month,day
4,200500,2020,09/07/2021,Avon,245 NEW ROAD,217640.0,400000.0,0.544100,Residential,Single Family,nan,nan,nan,nan,2021,07,09
6,20058,2020,06/01/2021,Barkhamsted,46 RATLUM MTN RD,203530.0,415000.0,0.490434,Residential,Single Family,nan,"2003 COLONIAL, 2140 SFLA, 2.99 AC",nan,nan,2021,01,06
9,2020360,2020,08/10/2021,Berlin,94 PERCIVAL AVE,140600.0,190790.0,0.736900,Residential,Single Family,nan,nan,nan,nan,2021,10,08
15,20062,2020,05/10/2021,Bolton,39 STONEHEDGE LN,189400.0,273750.0,0.691800,Residential,Single Family,nan,nan,nan,nan,2021,10,05
17,200400,2020,04/08/2021,Branford,460 MAIN ST,180500.0,355000.0,0.508400,Residential,Two Family,nan,nan,nan,nan,2021,08,04
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
997197,190310,2019,06/04/2020,Westport,11 FRAGRANT PINES COURT,410300.0,722000.0,0.568300,Single Family,Single Family,nan,nan,nan,nan,2020,04,06
997199,190365,2019,02/10/2020,West Hartford,191 HUNTER DRIVE,315420.0,380000.0,0.830100,Single Family,Single Family,nan,nan,nan,nan,2020,10,02
997205,190161,2019,03/02/2020,Newington,22 ROCKLEDGE DR,306000.0,456000.0,0.671100,Single Family,Single Family,nan,nan,nan,nan,2020,02,03
997207,190272,2019,08/03/2020,Wilton,145 WHIPSTICK RD,681870.0,1134708.0,0.600900,Single Family,Single Family,25 - Other,nan,"PER MLS CLOSING PRICE = $1,145,000",nan,2020,03,08


In [41]:
df['is_weekend']= df[['year', 'month', 'day']].astype(int).apply(lambda x: datetime.date(*x.values.tolist()).weekday(), axis=1)
df['is_weekend']= df['is_weekend'].apply(lambda x: 1 if x in (6, 7) else 0)
df['is_weekend']

4         0
6         0
9         0
15        0
17        0
         ..
997197    0
997199    0
997205    0
997207    1
997211    0
Name: is_weekend, Length: 385643, dtype: int64

In [39]:
df['is_weekend']

4         4
6         2
9         4
15        1
17        2
         ..
997197    0
997199    4
997205    0
997207    6
997211    4
Name: is_weekend, Length: 385643, dtype: int64

In [10]:
df['is_weekend']

4         4
6         2
9         4
15        1
17        2
         ..
997197    0
997199    4
997205    0
997207    6
997211    4
Name: is_weekend, Length: 385643, dtype: int64

In [12]:
df.corr(numeric_only= True)

,Serial Number,List Year,Assessed Value,Sale Amount,Sales Ratio,is_weekend
Serial Number,1.000000,0.086949,-0.001450,0.001049,-0.000375,-0.002949
List Year,0.086949,1.000000,0.034909,0.020230,-0.004364,-0.012982
Assessed Value,-0.001450,0.034909,1.000000,0.348077,0.006065,0.001425
Sale Amount,0.001049,0.020230,0.348077,1.000000,-0.000993,0.006154
Sales Ratio,-0.000375,-0.004364,0.006065,-0.000993,1.000000,0.002754
is_weekend,-0.002949,-0.012982,0.001425,0.006154,0.002754,1.000000


In [13]:
df['year_and_month']= df['year'] + '/' + df['month']

In [14]:
df['Sale Amount'].apply(lambda x: type(x))

4         <class 'float'>
6         <class 'float'>
9         <class 'float'>
15        <class 'float'>
17        <class 'float'>
               ...       
997197    <class 'float'>
997199    <class 'float'>
997205    <class 'float'>
997207    <class 'float'>
997211    <class 'float'>
Name: Sale Amount, Length: 385643, dtype: object

In [15]:
df_sales_stat= pd.DataFrame(df[['year_and_month', 'Assessed Value']].groupby('year_and_month').mean()['Assessed Value'])


In [16]:
df.columns

Index(['Serial Number', 'List Year', 'Date Recorded', 'Town', 'Address',
       'Assessed Value', 'Sale Amount', 'Sales Ratio', 'Property Type',
       'Residential Type', 'Non Use Code', 'Assessor Remarks', 'OPM remarks',
       'Location', 'year', 'month', 'day', 'is_weekend', 'year_and_month'],
      dtype='object')

In [17]:
fig= go.Figure(data= [
                        go.Scatter(x= df_sales_stat.index, y= df_sales_stat.iloc[:, 0], mode= 'markers+lines')

])
fig.show()

In [18]:
fig= go.Figure(data= [
                        go.Scatter(x= df_sales_stat.index, y= df_sales_stat.iloc[:, 0], mode= 'markers+lines')

])
fig.show()

In [19]:
set(df.columns)

{'Address',
 'Assessed Value',
 'Assessor Remarks',
 'Date Recorded',
 'List Year',
 'Location',
 'Non Use Code',
 'OPM remarks',
 'Property Type',
 'Residential Type',
 'Sale Amount',
 'Sales Ratio',
 'Serial Number',
 'Town',
 'day',
 'is_weekend',
 'month',
 'year',
 'year_and_month'}

In [20]:
df['Residential Type'].value_counts()

Residential Type
Single Family    170214
nan              151316
Condo             46102
Two Family        11659
Three Family       5414
Four Family         938
Name: count, dtype: int64

In [21]:
df

,Serial Number,List Year,Date Recorded,Town,Address,Assessed Value,Sale Amount,Sales Ratio,Property Type,Residential Type,Non Use Code,Assessor Remarks,OPM remarks,Location,year,month,day,is_weekend,year_and_month
4,200500,2020,09/07/2021,Avon,245 NEW ROAD,217640.0,400000.0,0.544100,Residential,Single Family,nan,nan,nan,nan,2021,07,09,4,2021/07
6,20058,2020,06/01/2021,Barkhamsted,46 RATLUM MTN RD,203530.0,415000.0,0.490434,Residential,Single Family,nan,"2003 COLONIAL, 2140 SFLA, 2.99 AC",nan,nan,2021,01,06,2,2021/01
9,2020360,2020,08/10/2021,Berlin,94 PERCIVAL AVE,140600.0,190790.0,0.736900,Residential,Single Family,nan,nan,nan,nan,2021,10,08,4,2021/10
15,20062,2020,05/10/2021,Bolton,39 STONEHEDGE LN,189400.0,273750.0,0.691800,Residential,Single Family,nan,nan,nan,nan,2021,10,05,1,2021/10
17,200400,2020,04/08/2021,Branford,460 MAIN ST,180500.0,355000.0,0.508400,Residential,Two Family,nan,nan,nan,nan,2021,08,04,2,2021/08
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
997197,190310,2019,06/04/2020,Westport,11 FRAGRANT PINES COURT,410300.0,722000.0,0.568300,Single Family,Single Family,nan,nan,nan,nan,2020,04,06,0,2020/04
997199,190365,2019,02/10/2020,West Hartford,191 HUNTER DRIVE,315420.0,380000.0,0.830100,Single Family,Single Family,nan,nan,nan,nan,2020,10,02,4,2020/10
997205,190161,2019,03/02/2020,Newington,22 ROCKLEDGE DR,306000.0,456000.0,0.671100,Single Family,Single Family,nan,nan,nan,nan,2020,02,03,0,2020/02
997207,190272,2019,08/03/2020,Wilton,145 WHIPSTICK RD,681870.0,1134708.0,0.600900,Single Family,Single Family,25 - Other,nan,"PER MLS CLOSING PRICE = $1,145,000",nan,2020,03,08,6,2020/03


In [22]:
df['Town'].drop_duplicates()

4                    Avon
6             Barkhamsted
9                  Berlin
15                 Bolton
17               Branford
               ...       
67998           Salisbury
68184      North Branford
68268         Marlborough
69790    North Stonington
69880           Voluntown
Name: Town, Length: 169, dtype: object

In [23]:
display(df)

,Serial Number,List Year,Date Recorded,Town,Address,Assessed Value,Sale Amount,Sales Ratio,Property Type,Residential Type,Non Use Code,Assessor Remarks,OPM remarks,Location,year,month,day,is_weekend,year_and_month
4,200500,2020,09/07/2021,Avon,245 NEW ROAD,217640.0,400000.0,0.544100,Residential,Single Family,nan,nan,nan,nan,2021,07,09,4,2021/07
6,20058,2020,06/01/2021,Barkhamsted,46 RATLUM MTN RD,203530.0,415000.0,0.490434,Residential,Single Family,nan,"2003 COLONIAL, 2140 SFLA, 2.99 AC",nan,nan,2021,01,06,2,2021/01
9,2020360,2020,08/10/2021,Berlin,94 PERCIVAL AVE,140600.0,190790.0,0.736900,Residential,Single Family,nan,nan,nan,nan,2021,10,08,4,2021/10
15,20062,2020,05/10/2021,Bolton,39 STONEHEDGE LN,189400.0,273750.0,0.691800,Residential,Single Family,nan,nan,nan,nan,2021,10,05,1,2021/10
17,200400,2020,04/08/2021,Branford,460 MAIN ST,180500.0,355000.0,0.508400,Residential,Two Family,nan,nan,nan,nan,2021,08,04,2,2021/08
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
997197,190310,2019,06/04/2020,Westport,11 FRAGRANT PINES COURT,410300.0,722000.0,0.568300,Single Family,Single Family,nan,nan,nan,nan,2020,04,06,0,2020/04
997199,190365,2019,02/10/2020,West Hartford,191 HUNTER DRIVE,315420.0,380000.0,0.830100,Single Family,Single Family,nan,nan,nan,nan,2020,10,02,4,2020/10
997205,190161,2019,03/02/2020,Newington,22 ROCKLEDGE DR,306000.0,456000.0,0.671100,Single Family,Single Family,nan,nan,nan,nan,2020,02,03,0,2020/02
997207,190272,2019,08/03/2020,Wilton,145 WHIPSTICK RD,681870.0,1134708.0,0.600900,Single Family,Single Family,25 - Other,nan,"PER MLS CLOSING PRICE = $1,145,000",nan,2020,03,08,6,2020/03


In [24]:
df_expensive_towns= df[['Town', 'Assessed Value', 'Sale Amount', 'Sales Ratio']].groupby('Town').mean().sort_values(by= 'Sale Amount', ascending= True)
df_expensive_towns= df_expensive_towns.iloc[-12:].sort_values(by= 'Sale Amount', ascending= True)
# fig= go.Figure(data= [
#                             go.Histogram(x= df_expensive_towns.index, y= df_expensive_towns['Sales Ratio'])

# ])
fig = go.Figure([go.Bar(y= df_expensive_towns['Sale Amount'], x=list(df_expensive_towns.index))])
fig.show()

In [25]:
df_expensive_towns['Assessed Value']

Town
Fairfield     4.558580e+05
Ridgefield    4.766150e+05
Roxbury       4.687955e+05
Washington    4.907229e+05
Willington    1.663151e+05
Wilton        5.863877e+05
Weston        6.063468e+05
Stamford      5.058892e+05
Westport      8.604188e+05
New Canaan    1.121975e+06
Darien        9.626772e+05
Greenwich     2.533998e+06
Name: Assessed Value, dtype: float64

In [27]:
df

,Serial Number,List Year,Date Recorded,Town,Address,Assessed Value,Sale Amount,Sales Ratio,Property Type,Residential Type,Non Use Code,Assessor Remarks,OPM remarks,Location,year,month,day,is_weekend,year_and_month
4,200500,2020,09/07/2021,Avon,245 NEW ROAD,217640.0,400000.0,0.544100,Residential,Single Family,nan,nan,nan,nan,2021,07,09,4,2021/07
6,20058,2020,06/01/2021,Barkhamsted,46 RATLUM MTN RD,203530.0,415000.0,0.490434,Residential,Single Family,nan,"2003 COLONIAL, 2140 SFLA, 2.99 AC",nan,nan,2021,01,06,2,2021/01
9,2020360,2020,08/10/2021,Berlin,94 PERCIVAL AVE,140600.0,190790.0,0.736900,Residential,Single Family,nan,nan,nan,nan,2021,10,08,4,2021/10
15,20062,2020,05/10/2021,Bolton,39 STONEHEDGE LN,189400.0,273750.0,0.691800,Residential,Single Family,nan,nan,nan,nan,2021,10,05,1,2021/10
17,200400,2020,04/08/2021,Branford,460 MAIN ST,180500.0,355000.0,0.508400,Residential,Two Family,nan,nan,nan,nan,2021,08,04,2,2021/08
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
997197,190310,2019,06/04/2020,Westport,11 FRAGRANT PINES COURT,410300.0,722000.0,0.568300,Single Family,Single Family,nan,nan,nan,nan,2020,04,06,0,2020/04
997199,190365,2019,02/10/2020,West Hartford,191 HUNTER DRIVE,315420.0,380000.0,0.830100,Single Family,Single Family,nan,nan,nan,nan,2020,10,02,4,2020/10
997205,190161,2019,03/02/2020,Newington,22 ROCKLEDGE DR,306000.0,456000.0,0.671100,Single Family,Single Family,nan,nan,nan,nan,2020,02,03,0,2020/02
997207,190272,2019,08/03/2020,Wilton,145 WHIPSTICK RD,681870.0,1134708.0,0.600900,Single Family,Single Family,25 - Other,nan,"PER MLS CLOSING PRICE = $1,145,000",nan,2020,03,08,6,2020/03


In [37]:
df['Address'].apply(lambda x: ' '.join(x.split(' ')[1:]))

4                     NEW ROAD
6                RATLUM MTN RD
9                 PERCIVAL AVE
15               STONEHEDGE LN
17                     MAIN ST
                  ...         
997197    FRAGRANT PINES COURT
997199            HUNTER DRIVE
997205            ROCKLEDGE DR
997207            WHIPSTICK RD
997211              BYSTREK DR
Name: Address, Length: 385643, dtype: object

In [ ]:
df